# Investigate problems related to work not_work data

## Gather data

In [1]:
import constants
import pandas as pd
import os
from IPython.display import display
import json
import numpy as np
path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))
input_directory = parent + '/'+constants.output_data_segment_keyword_matrix
segment_df = pd.read_csv(input_directory + constants.output_segment_keyword_matrix_document_index)
int_codes = segment_df['IntCode'].to_list()
with open(parent+'/'+constants.output_data_segment_keyword_matrix + "metadata_partitions.json") as read_file:
        metadata_partitions = json.load(read_file)
input_folder = parent+'/'+constants.input_data
input_file = constants.input_files_biodata_birkenau
df_biodata = pd.read_csv(input_folder+input_file)
df_biodata = df_biodata[df_biodata.IntCode.isin(int_codes)]

Load the very original segment data

In [2]:
input_directory = constants.input_data
input_files = constants.input_files_segments
input_files = [parent+'/'+input_directory+i for i in input_files]
df = pd.concat([pd.read_csv(el) for el in input_files])
df = df.reset_index()

Load the segment keyword matrix

In [3]:
 # Load the input data
input_directory = parent +'/'+constants.output_data_segment_keyword_matrix
# Read the segment index term matrix
data = np.load(input_directory + constants.output_segment_keyword_matrix_data_file.replace('.txt', '.npy'),  allow_pickle=True)
# Read the column index (index terms) of the matrix above
features_df = pd.read_csv(input_directory + constants.output_segment_keyword_matrix_feature_index)

## Check the original data

Get the int code of those who worked and who did not work

Not work first

In [4]:
int_codes_notwork = []
for element in metadata_partitions['notwork']:
    int_codes_notwork.append(int_codes[element])
    
    

Choose randomly from the list

In [5]:
import random
not_work_random = random.sample(int_codes_notwork,100)

Check if contains work

In [6]:
#df[df.IntCode==not_work_random].KeywordLabel.value_counts()
for element in not_work_random:
    random_df = df[df.IntCode==element]
    if random_df[random_df['KeywordLabel'].str.contains('labor')].size >0:
        print (element)
        print (random_df[random_df['KeywordLabel'].str.contains('labor')]['KeywordLabel'])


41188
802829          camp forced labor
802834    forced labor conditions
Name: KeywordLabel, dtype: object
466
11460    camp forced labor
Name: KeywordLabel, dtype: object
36992
733749    camp forced labor
Name: KeywordLabel, dtype: object
1521
40895    camp forced labor
Name: KeywordLabel, dtype: object
9026
213173    camp forced labor
Name: KeywordLabel, dtype: object
38957
766166    camp forced labor
Name: KeywordLabel, dtype: object


Work second

In [7]:
int_codes_work = []
for element in metadata_partitions['work']:
    int_codes_work.append(int_codes[element])
    
    

Choose randomly from the list

In [8]:
import random
work_random = random.sample(int_codes_work,100)

Check if contains work

In [9]:
#df[df.IntCode==not_work_random].KeywordLabel.value_counts()
for element in work_random:
    random_df = df[df.IntCode==element]
    if random_df[random_df['KeywordLabel'].str.contains('labor')].size >0:
        print (element)
        print (random_df[random_df['KeywordLabel'].str.contains('labor')]['KeywordLabel'])


14878
330416    camp construction forced labor
330417                 camp forced labor
Name: KeywordLabel, dtype: object
33563
675050                       armament forced labor
675052                           camp forced labor
675057                       armament forced labor
675059                           camp forced labor
675061    German forced labor civilian supervisors
Name: KeywordLabel, dtype: object
272
7720                         camp forced labor
7722    personal property sorting forced labor
Name: KeywordLabel, dtype: object
54679
1045254                 camp forced labor
1045257    road construction forced labor
Name: KeywordLabel, dtype: object
53590
1014892          camp forced labor
1014925       factory forced labor
1014926    forced labor conditions
Name: KeywordLabel, dtype: object
53607
1015655    arts-related forced labor
Name: KeywordLabel, dtype: object
8083
191080     camp daily services forced labor
191081                    camp forced labor
191120      

9902
231018               camp forced labor
231019       construction forced labor
231046    forced labor business owners
Name: KeywordLabel, dtype: object
51069
971866                    camp forced labor
971871                  quarry forced labor
971879    textiles and garment forced labor
Name: KeywordLabel, dtype: object
51631
985619    camp daily services forced labor
985620                   camp forced labor
Name: KeywordLabel, dtype: object
44316
848345    personal property sorting forced labor
848347                       quarry forced labor
848348         textiles and garment forced labor
Name: KeywordLabel, dtype: object
13532
302726                         camp forced labor
302729                       quarry forced labor
302752          camp daily services forced labor
302753                         camp forced labor
302763                         camp forced labor
302766    personal property sorting forced labor
Name: KeywordLabel, dtype: object
20820
452987             

## Check the segment keyword matrix

In [10]:
index_social_bond = features_df.KeywordLabel.to_list().index('social bonds')

First not work

In [11]:
seg_keyword = np.take(data,metadata_partitions['notwork'])

In [12]:
assert len(seg_keyword) == len(int_codes_notwork)

Check how many times a person mentions social bonds on average

In [13]:
counts = []
for element in seg_keyword:
    counts.append(np.nonzero(element[:,index_social_bond])[0].shape[0])
df_counts = pd.DataFrame(counts)

Percentage of those who mention it at least once

In [14]:
len(df_counts[df_counts[0]>0])/len(df_counts)*100

14.968279143536876

In [15]:
len(df_counts[df_counts[0]>0])

755

In [16]:
percentage_mentioning_at_least_one_count_notwork=len(df_counts[df_counts[0]>0])/len(df_counts)*100

Total number of mentioning social bonds

In [17]:
df_counts[0].sum()

1013

In [18]:
total_n_mentioning_social_bonds_notwork = df_counts[0].sum()

Then check work

In [19]:
seg_keyword = np.take(data,metadata_partitions['work'])

In [20]:
assert len(seg_keyword) == len(int_codes_work)

Check how many times a person mentions social bonds on average

In [21]:
counts = []
for element in seg_keyword:
    counts.append(np.nonzero(element[:,index_social_bond])[0].shape[0])
df_counts = pd.DataFrame(counts)

In [22]:
df_counts.describe()

,0
count,1584.000000
mean,0.455808
std,0.965909
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,10.000000


Percentage of those who mention it at least once

In [23]:
percentage_mentioning_at_least_one_count_work=len(df_counts[df_counts[0]>0])/len(df_counts)*100

In [24]:
len(df_counts[df_counts[0]>0])

434

Total number of mentioning social bonds

In [25]:
df_counts[0].sum()

722

In [26]:
total_n_mentioning_social_bonds_work = df_counts[0].sum()

## Check the markov training

In [27]:
from markov_modelling import markov_utils as mu

First those who worked

In [28]:
seg_keyword = np.take(data,metadata_partitions['work'])
trajs = mu.estimate_fuzzy_trajectories(seg_keyword)

Check how many times social bonds occur

In [29]:
n=0
row_occurrences_work = 0
for traj in trajs:
    if index_social_bond in traj:
        n+=1
        occurrences = np.count_nonzero(traj == index_social_bond)
        row_occurrences_work = row_occurrences_work+occurrences

    
    

Total number of trajectories where social bonds mentioned at least once  

In [30]:
print (n/len(trajs))

0.1905050505050505


In [31]:
percentage_traj_mentionining_once_to_total_work = (n/len(trajs))*100

In [32]:
n/len(trajs)

0.1905050505050505

Estimate the Markov model1

In [33]:
msm_work = mu.estimate_markov_model_from_trajectories(trajs)

In [188]:
index = features_df.KeywordLabel.to_list().index('living conditions')
print (index)
initial = np.zeros(111)
initial[index]=1
result = msm_work.propagate(initial,1)[::-1]
print(np.argsort(result))
print(features_df.KeywordLabel.to_list()[np.argsort(result)[-1]])

64
[  8  65  83  18 105  29  73  67  25  58  36  87  41 101  47 107  95 106
  31  93   3  71  96  68  23  79  28  61  66  38  92   6  24  15  42  54
  74  57  48  50  35 108  19   7  53   2  27  14   0  32 100  94  17  76
  49  97  37  59  26 109  56  63  34  99  80  11  86  82  55  12  81   4
  70  10  98  43  77  64  45 104  22  85  91  30  78  21  44  90  20  88
  72  84  51  39  33  75 103   5  69  13 102   1   9  89  40  16 110  52
  62  60  46]
food acquisition


Then those who did not work

In [34]:
seg_keyword = np.take(data,metadata_partitions['notwork'])
trajs = mu.estimate_fuzzy_trajectories(seg_keyword)
msm_notwork = mu.estimate_markov_model_from_trajectories(trajs)

Check how many times social bonds occur

In [35]:
n=0
row_occurrences_notwork = 0
for traj in trajs:
    if index_social_bond in traj:
        n+=1
        occurrences = np.count_nonzero(traj == index_social_bond)
        row_occurrences_notwork = row_occurrences_notwork+occurrences

    

Total number of trajectories where social bonds mentioned at least once

In [36]:
print (n)

13009


In [37]:
print (n/len(trajs))

0.10316415543219667


In [38]:
percentage_traj_mentionining_once_to_total_notwork = n/len(trajs)*100

In [39]:
ratio_mentioning_trajs_total_trajs_not_work = n/len(trajs)

In [40]:
msm_notwork.pi.shape

(110,)

In [41]:
msm_work.pi.shape

(111,)

In [42]:
index_social_bond

97

Compare stationary probabilities

In [43]:
pi_social_bonds_work=msm_work.pi[msm_work._full2active[index_social_bond]]

In [44]:
pi_social_bonds_notwork=msm_notwork.pi[msm_notwork._full2active[index_social_bond]]

# Problem

- 27% of the working people mention social bonds at least once
- 15% of the non working people mention social bonds at least once
- this explains why the Fisher test tells that 'social bonds' is more associated with the working people
- 10.3% of the trajectories of those who did not work include 'social bonds' at least once
- 18.8% of the trajectories of those who worked include 'social bonds' at least once
- The ratios are more or less the same
 
**BUT**

- but when stationary probabilities are compared, the stationary probability of social bonds in the non working is higher than the stationary prob of work

In [45]:
percentage_mentioning_at_least_one_count_work

27.3989898989899

In [46]:
percentage_mentioning_at_least_one_count_notwork

14.968279143536876

In [47]:
percentage_traj_mentionining_once_to_total_notwork

10.316415543219668

In [48]:
percentage_traj_mentionining_once_to_total_work

19.050505050505052

In [49]:
percentage_mentioning_at_least_one_count_work > percentage_mentioning_at_least_one_count_notwork


True

In [50]:
percentage_traj_mentionining_once_to_total_work > percentage_traj_mentionining_once_to_total_notwork

True

In [51]:
percentage_mentioning_at_least_one_count_work / percentage_mentioning_at_least_one_count_notwork

1.8304702655696032

In [52]:
percentage_traj_mentionining_once_to_total_work / percentage_traj_mentionining_once_to_total_notwork

1.8466205602803343

In [53]:
pi_social_bonds_notwork > pi_social_bonds_work

True

In [54]:
pi_social_bonds_notwork / pi_social_bonds_work

1.124636090653183